In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# pip install evalml

In [4]:
!pip install pycaret

     |████████████████████████████████| 301 kB 13.3 MB/s 
     |████████████████████████████████| 2.0 MB 44.9 MB/s 
     |████████████████████████████████| 1.7 MB 46.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 47.3 MB/s 
     |████████████████████████████████| 6.8 MB 19.8 MB/s 
     |████████████████████████████████| 15.6 MB 47.6 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 114 kB 53.0 MB/s 
     |████████████████████████████████| 261 kB 50.4 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 1.3 MB 43.4 MB/s 
     |████████████████████████████████| 3.1 MB 48.5 MB/s 
     |████████████████████████████████| 10.9 MB 44.1 MB/s 
     |████████████████████████████████| 675 kB 46.0 MB/s 
     |███████████████

In [5]:
# Loading the Data
train=pd.read_csv('/content/drive/MyDrive/DataSets/Data/Cascade/train.csv')
test=pd.read_csv('/content/drive/MyDrive/DataSets/Data/Cascade/test.csv')
sample_submission=pd.read_csv('/content/drive/MyDrive/DataSets/Data/Cascade/sample_submission.csv')

In [6]:
train.head()

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [7]:
train.shape

(450000, 20)

In [8]:
test.head(1000)

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.658500,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.070900,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.388400,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.903900,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.827500,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021-02-06 11:04:57,131226,2021-02-06 00:00:00,2021-02-06 11:04:57,2021-02-06 11:05:11,11015,0.748000,3.24,310.0,308.0,2.0,902.0,NaN,NaN,NaN,247.250000
996,2021-02-06 11:04:57,131227,2021-02-06 00:00:00,2021-02-06 11:04:58,2021-02-06 11:05:57,2394,2.627200,3.84,476.0,475.0,1.0,1540.0,NaN,NaN,NaN,410.316667
997,2021-02-06 11:04:58,131228,2021-02-06 00:00:00,2021-02-06 11:04:59,2021-02-06 11:05:30,3780,1.998500,6.97,216.0,216.0,0.0,237.0,NaN,NaN,NaN,15.033333
998,2021-02-06 11:05:03,131229,2021-02-06 00:00:00,2021-02-06 11:05:31,2021-02-06 11:05:36,1838,0.122107,3.25,41.0,40.0,1.0,836.0,NaN,NaN,NaN,316.650000


train.csv: This file contains one row for each order created in a specific time frame. The columns are:

- order_id : unique id for each order
-order_time: time of the creation of order by the client
- order_date : date of the order
-allot_time: time of allocation of order to the rider
- accept_time: time of acceptance of the order by the - rider (if available)
- pickup_time: time of pickup of the order (if available)
- delivered_time: time of delivery of the order (if available)
- cancelled_time: time of cancellation of order (if the order was cancelled)
- cancelled: whether the order was cancelled
- rider_id: unique id for each rider
- first_mile_distance: road distance from rider’s - - - location to the pickup location
- last_mile_distance: road distance from pickup - - - - location to the delivery location
- allotted_orders: total number of orders allotted to the rider in the 30 days before (not including) order_date
- delivered_orders: total number of orders delivered by the rider in the 30 days before (not including) order_date
- undelivered_orders: total number of orders allotted to but not delivered by the rider (i.e. cancelled) in the 30 days before (not including) order_date
- lifetime_order_count: total number of orders delivered by the rider at any time before order_date
- reassigned_order: whether the order was reassigned to this rider
- reassignment_method: if the order was reassigned, whether the reassignment was done manually (by the ops team) or automatically
- reassignment_reason: a more detailed reason for the reassignment
- session_time: total time the rider had been online on order_date before order_time

In [9]:
# EDA nathi jarur
# Data Pre Processing and Feature Engineering
# Cleaning the Data
train.isnull().sum()
# Strategy
# The 95 rows where accept time is not given - REMOVE
# The 1497 Pickup Time not given - Replace with time +1,2 mins acc. to trend to the time order was place/ accepted or whatever is available
# The 3148 Delivered time not given must be replace with an average time based on last mile dist, or by Regression whatever suits best
# Cancelled 1 value to be replaced with 1
# Alloted Orders to be replaced by Average
# Delivered Orders ~ 97% of Alloted Orders
# Undelivered Orders -> Total orders - Delivered orders  or Total -  98% of total if delivered not give, else REMOVE
# Lifetime Order Count - Make 103% of Delivered Orders if Available otherwise Remove
# Session Time -> Replace with mean
# Reassigned Method -> 0 Empty, 1 Auto, 2 Manual
# Reassignment reason -> 0,1,2 in Case of Empty
# Cancelled waala ma sarkhu karvu padse Feature Engineering
# Create an Ensemble Model

order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                157
pickup_time               2421
delivered_time            5218
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16948
delivered_orders         17341
cancelled                    0
undelivered_orders       17341
lifetime_order_count        53
reassignment_method     436256
reassignment_reason     436247
reassigned_order        436247
session_time              3675
cancelled_time          444782
dtype: int64

In [10]:
train.dropna(subset=['pickup_time'], inplace=True)
# test.dropna(subset=['accept_time'], inplace=True)
train.dropna(subset=['delivered_time'], inplace=True)

In [11]:
train.dropna(subset=['accept_time'], inplace=True)
test.dropna(subset=['accept_time'], inplace=True)

In [12]:
train.dtypes

order_time               object
order_id                  int64
order_date               object
allot_time               object
accept_time              object
pickup_time              object
delivered_time           object
rider_id                  int64
first_mile_distance     float64
last_mile_distance      float64
alloted_orders          float64
delivered_orders        float64
cancelled                 int64
undelivered_orders      float64
lifetime_order_count    float64
reassignment_method      object
reassignment_reason      object
reassigned_order        float64
session_time            float64
cancelled_time           object
dtype: object

In [13]:
train.drop(['order_date'], axis = 1,inplace=True)
test.drop(['order_date'], axis = 1,inplace=True)

In [14]:
train.head()

,order_time,order_id,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


In [15]:
train['order_time']=train['order_time'].str[10:]
train['allot_time']=train['allot_time'].str[10:]
train['accept_time']=train['accept_time'].str[10:]
train['pickup_time']=train['pickup_time'].str[10:]
train['delivered_time']=train['delivered_time'].str[10:]
test['order_time']=test['order_time'].str[10:]
test['allot_time']=test['allot_time'].str[10:]
test['accept_time']=test['accept_time'].str[10:]
# test['pickup_time']=test['pickup_time'].str[10:]
# test['delivered_time']=test['delivered_time'].str[10:]

In [16]:
train.head()

,order_time,order_id,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,02:21:35,556753,02:21:59,02:22:08,02:32:51,02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,02:33:16,556754,02:33:57,02:34:45,02:50:25,03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,02:39:49,556755,02:39:57,02:40:13,02:56:00,03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,02:47:53,556756,02:48:25,02:49:06,03:21:51,03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,03:06:30,556757,03:07:21,03:07:57,03:31:38,04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN


- Durations are more Important not a Time Series Data
- Day of the Week is More Important and not the Date
- Time of the day is needed as a Label Encoding
- Rider Id is immaterial but check for duplicates
- Last Mile and First Mile Distance is important
- Cancelled - Decision is obviously important 
- Order ID is not important
- Accept time not imp, Order Time- Accept time duration needed
- Accept Time - Pickup Time and Order Time - Pickup Time Imp
- Time of cancel from all the checkpoints needed
- Delivered/Alloted ratio
- Undelivered / Alloted ratio
- Reassigned order 0 or 1
- Reassigned Method One Hot Encoding
- Session Time As it is

In [17]:
# train['order_time']=str(train['order_time'])
# train.order_time.dtype

In [18]:
duplicate = train[train.duplicated('rider_id')]
duplicate

,order_time,order_id,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
25,03:34:09,556778,03:35:47,03:36:13,03:45:48,04:12:32,2338,2.3211,5.61,35.0,34.0,0,1.0,116.0,NaN,NaN,NaN,3.883333,NaN
35,03:37:37,556788,03:38:32,03:38:51,03:53:55,04:27:04,18117,1.6622,5.89,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,67.616667,NaN
40,03:38:40,556793,03:39:06,03:39:45,04:08:09,04:24:12,2382,2.1271,2.40,229.0,227.0,0,2.0,24022.0,NaN,NaN,NaN,8.666667,NaN
45,03:39:49,556798,03:40:02,03:41:02,03:58:15,04:34:10,1440,1.3045,5.24,306.0,303.0,0,3.0,3868.0,NaN,NaN,NaN,5.566667,NaN
51,03:42:10,556804,03:42:46,03:43:58,04:09:08,04:40:00,3245,2.2788,3.81,305.0,303.0,0,2.0,833.0,NaN,NaN,NaN,0.433333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,10:03:16,130226,10:03:44,10:04:14,10:27:29,10:44:08,1006,0.5789,0.19,4.0,4.0,0,0.0,127.0,NaN,NaN,NaN,369.516667,NaN
449996,10:03:17,130227,10:03:18,10:04:34,10:22:17,10:31:43,279,1.9863,1.19,81.0,81.0,0,0.0,105.0,NaN,NaN,NaN,239.133333,NaN
449997,10:03:18,130228,10:04:06,10:04:39,10:19:06,10:26:56,3161,1.5944,1.61,28.0,28.0,0,0.0,1488.0,NaN,NaN,NaN,204.150000,NaN
449998,10:03:19,130229,10:03:19,10:05:41,10:20:39,10:30:41,9396,2.8939,4.68,72.0,72.0,0,0.0,105.0,NaN,NaN,NaN,65.583333,NaN


In [19]:
# train['order_time'] = pd.Series([val.time() for val in train['order_time']])
# train['order_time'] = pd.to_datetime(train['order_time'],format).apply(lambda x: x.time())

In [20]:
train.isnull().sum()

order_time                   0
order_id                     0
allot_time                   0
accept_time                  0
pickup_time                  0
delivered_time               0
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           16299
delivered_orders         16486
cancelled                    0
undelivered_orders       16486
lifetime_order_count         2
reassignment_method     431757
reassignment_reason     431751
reassigned_order        431751
session_time              3531
cancelled_time          444782
dtype: int64

In [21]:
test.isnull().sum()

order_time                   0
order_id                     0
allot_time                   0
accept_time                  0
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders            4768
delivered_orders          4879
undelivered_orders        4879
lifetime_order_count       778
reassignment_method     140148
reassignment_reason     140145
reassigned_order        140145
session_time              5050
dtype: int64

In [22]:
# Alloted to be replace by the Average
train.drop(['alloted_orders'], axis = 1,inplace=True)
test.drop(['alloted_orders'], axis = 1,inplace=True)
train.drop(['delivered_orders'], axis = 1,inplace=True)
test.drop(['delivered_orders'], axis = 1,inplace=True)
train.drop(['undelivered_orders'], axis = 1,inplace=True)
test.drop(['undelivered_orders'], axis = 1,inplace=True)
train.drop(['lifetime_order_count'], axis = 1,inplace=True)
test.drop(['lifetime_order_count'], axis = 1,inplace=True)

In [23]:
train.isnull().sum()

order_time                  0
order_id                    0
allot_time                  0
accept_time                 0
pickup_time                 0
delivered_time              0
rider_id                    0
first_mile_distance         0
last_mile_distance          0
cancelled                   0
reassignment_method    431757
reassignment_reason    431751
reassigned_order       431751
session_time             3531
cancelled_time         444782
dtype: int64

In [24]:
train.reassignment_method.unique()


array([nan, 'auto', 'manual'], dtype=object)

In [25]:
train.reassignment_reason.unique()


array([nan, 'Reassignment Request from SE portal.',
       'Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket',
       'Reassign'], dtype=object)

In [26]:
train.reassigned_order.unique()

array([nan,  1.])

In [27]:
train.drop(['cancelled_time'], axis = 1,inplace=True)

In [28]:
train.drop(['session_time'], axis = 1,inplace=True)

In [29]:
# One Hot Encoding and Imputing Lst Three
train['reassignment_method']=train['reassignment_method'].fillna('not_reassigned')

In [30]:
train['reassignment_reason']=train['reassignment_reason'].fillna('no_reason')

In [31]:
train['reassigned_order']=train['reassigned_order'].fillna(0)

In [32]:
train.isnull().sum()

order_time             0
order_id               0
allot_time             0
accept_time            0
pickup_time            0
delivered_time         0
rider_id               0
first_mile_distance    0
last_mile_distance     0
cancelled              0
reassignment_method    0
reassignment_reason    0
reassigned_order       0
dtype: int64

In [33]:
# Convert to date time and Add Durations to j mel padse
# import preprocessing from sklearn
from sklearn.preprocessing import LabelEncoder
# creating instance of labelencoder
labelencoder = LabelEncoder()
reassignment_methods=['not_reassigned', 'auto', 'manual']
reassignment_reasons=['no_reasons', 'Reassignment Request from SE portal.',
       'Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket',
       'Reassign']
train_one_hot=train[['reassignment_method','reassignment_reason']]
train_one_hot=pd.get_dummies(train_one_hot)
train_one_hot.head()

,reassignment_method_auto,reassignment_method_manual,reassignment_method_not_reassigned,reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket,reassignment_reason_Reassign,reassignment_reason_Reassignment Request from SE portal.,reassignment_reason_no_reason
0,0,0,1,0,0,0,1
1,0,0,1,0,0,0,1
2,0,0,1,0,0,0,1
3,0,0,1,0,0,0,1
4,0,0,1,0,0,0,1


In [34]:
train['reassignment_method_auto'] = train_one_hot['reassignment_method_auto']
train['reassignment_method_manual'] = train_one_hot['reassignment_method_manual']
train['reassignment_method_not_reassigned'] = train_one_hot['reassignment_method_not_reassigned']
train['reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket'] = train_one_hot['reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket']
train['reassignment_reason_Reassign'] = train_one_hot['reassignment_reason_Reassign']
train['reassignment_reason_Reassignment Request from SE portal.'] = train_one_hot['reassignment_reason_Reassignment Request from SE portal.']
train['reassignment_reason_no_reason'] = train_one_hot['reassignment_reason_no_reason']


In [35]:
train.drop(['reassignment_method','reassignment_reason'],axis=1,inplace=True)

In [36]:
# Converting to Datetime and also getting durations
train.dtypes

order_time                                                                                           object
order_id                                                                                              int64
allot_time                                                                                           object
accept_time                                                                                          object
pickup_time                                                                                          object
delivered_time                                                                                       object
rider_id                                                                                              int64
first_mile_distance                                                                                 float64
last_mile_distance                                                                                  float64
cancelled                   

In [37]:
train.head()

,order_time,order_id,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,cancelled,reassigned_order,reassignment_method_auto,reassignment_method_manual,reassignment_method_not_reassigned,reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket,reassignment_reason_Reassign,reassignment_reason_Reassignment Request from SE portal.,reassignment_reason_no_reason
0,02:21:35,556753,02:21:59,02:22:08,02:32:51,02:49:47,11696,1.5666,2.65,0,0.0,0,0,1,0,0,0,1
1,02:33:16,556754,02:33:57,02:34:45,02:50:25,03:11:15,18117,2.5207,2.76,0,0.0,0,0,1,0,0,0,1
2,02:39:49,556755,02:39:57,02:40:13,02:56:00,03:12:46,18623,2.2074,4.80,0,0.0,0,0,1,0,0,0,1
3,02:47:53,556756,02:48:25,02:49:06,03:21:51,03:41:05,15945,2.1894,6.38,0,0.0,0,0,1,0,0,0,1
4,03:06:30,556757,03:07:21,03:07:57,03:31:38,04:00:15,17589,2.7870,4.01,0,0.0,0,0,1,0,0,0,1


In [38]:
# train['order_time'] = train['order_time'].astype('datetime64[ns]')
train['order_time_t'] = pd.to_datetime(train['order_time'], errors = 'coerce')
train[train['order_time_t'].isna()]

,order_time,order_id,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,cancelled,reassigned_order,reassignment_method_auto,reassignment_method_manual,reassignment_method_not_reassigned,reassignment_reason_Auto Reassignment basis Inaction. coreengine.tasks.repush_order_to_aa_bucket,reassignment_reason_Reassign,reassignment_reason_Reassignment Request from SE portal.,reassignment_reason_no_reason,order_time_t
0,02:21:35,556753,02:21:59,02:22:08,02:32:51,02:49:47,11696,1.5666,2.65,0,0.0,0,0,1,0,0,0,1,NaT
1,02:33:16,556754,02:33:57,02:34:45,02:50:25,03:11:15,18117,2.5207,2.76,0,0.0,0,0,1,0,0,0,1,NaT
2,02:39:49,556755,02:39:57,02:40:13,02:56:00,03:12:46,18623,2.2074,4.80,0,0.0,0,0,1,0,0,0,1,NaT
3,02:47:53,556756,02:48:25,02:49:06,03:21:51,03:41:05,15945,2.1894,6.38,0,0.0,0,0,1,0,0,0,1,NaT
4,03:06:30,556757,03:07:21,03:07:57,03:31:38,04:00:15,17589,2.7870,4.01,0,0.0,0,0,1,0,0,0,1,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,10:03:16,130226,10:03:44,10:04:14,10:27:29,10:44:08,1006,0.5789,0.19,0,0.0,0,0,1,0,0,0,1,NaT
449996,10:03:17,130227,10:03:18,10:04:34,10:22:17,10:31:43,279,1.9863,1.19,0,0.0,0,0,1,0,0,0,1,NaT
449997,10:03:18,130228,10:04:06,10:04:39,10:19:06,10:26:56,3161,1.5944,1.61,0,0.0,0,0,1,0,0,0,1,NaT
449998,10:03:19,130229,10:03:19,10:05:41,10:20:39,10:30:41,9396,2.8939,4.68,0,0.0,0,0,1,0,0,0,1,NaT


In [39]:
df_temp = train[train['order_time_t'].isna()]
train.loc[df_temp.index, 'order_time']= df_temp['order_time'] 

In [40]:
train.head()
train.drop(['order_time_t'],axis=1,inplace=True)

In [ ]:
train['allot_time_t'] = pd.to_datetime(train['allot_time'], errors = 'coerce')
train[train['allot_time_t'].isna()]
df_temp = train[train['allot_time_t'].isna()]
train.loc[df_temp.index, 'allot_time']= df_temp['allot_time'] 
train.drop(['allot_time_t'],axis=1,inplace=True)

train['accept_time_t'] = pd.to_datetime(train['accept_time'], errors = 'coerce')
train[train['accept_time_t'].isna()]
df_temp = train[train['accept_time_t'].isna()]
train.loc[df_temp.index, 'accept_time']= df_temp['accept_time'] 
train.drop(['accept_time_t'],axis=1,inplace=True)

train['pickup_time_t'] = pd.to_datetime(train['pickup_time'], errors = 'coerce')
train[train['pickup_time_t'].isna()]
df_temp = train[train['pickup_time_t'].isna()]
train.loc[df_temp.index, 'pickup_time']= df_temp['pickup_time'] 
train.drop(['pickup_time_t'],axis=1,inplace=True)


train['delivered_time_t'] = pd.to_datetime(train['delivered_time'], errors = 'coerce')
train[train['delivered_time_t'].isna()]
df_temp = train[train['delivered_time_t'].isna()]
train.loc[df_temp.index, 'delivered_time']= df_temp['delivered_time'] 
train.drop(['delivered_time_t'],axis=1,inplace=True)



In [ ]:
# Now we will add durations by taking time Differences and then drop the Columns and then Build Models
train.head()

In [ ]:
# # Take Differences of two Times
# train['diff'] = train['allot_time'] - train['order_time']
# train['diff']

In [ ]:
from pycaret.classification import *
s= setup(data = train, target = 'cancelled')